<a href="https://colab.research.google.com/github/YunhaoZZ/CS6347Project_GamePredictor/blob/main/Steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!unzip /content/steam_games.csv.zip

Archive:  /content/steam_games.csv.zip
replace steam_games.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: steam_games.csv         


# Data Preprocess

In [77]:
import pandas
import numpy as np
from collections import Counter

In [78]:
def preprocess(path):
  fields = ['App ID', 'Tags', 'Positive Reviews','Negative Reviews']
  df = pandas.read_csv(path, usecols=fields, delimiter=';')
  return df

In [79]:
def getDataset(path): 
  #path="steam_games.csv"
  num=500
  df=preprocess(path)
  df = df.dropna()
  df = df.reset_index(drop=True)
  # print(df['Tags'].head())

  # record tags into a dictionary
  tags = {}
  gTag = []
  for row in df['Tags']:
    tag_pairs = row.split(',')
    temp = []
    for tag_pair in tag_pairs:
      tag_items = tag_pair.split(':')
      if tag_items[0] not in tags.keys():
        tags[tag_items[0]] = 1
      else:
        tags[tag_items[0]] += 1
      temp.append(tag_items[0])
    gTag.append(temp)

  tag_counter = Counter(tags)
  tags = tag_counter.most_common(num)

  tag_id = {}
  for idx, tag in enumerate(tags):
    tag_id[idx] = tag[0]

  df = df.drop(columns=['Tags'])
  df['Tags'] = gTag

  rm = []
  y = []
  for idx, row in enumerate(df['Tags']):
    # temp = np.zeros(500)
    temp = [0 for _ in range(num)]
    for tag in row:
      if tag in tag_id.values():
        temp[list(tag_id.keys())[list(tag_id.values()).index(tag)]] = 1
    y.append(temp)
    if 1 not in temp:
      rm.append(idx)

  # y = np.array(y)
  df['y'] = y

  df = df.drop(rm)
  df = df.reset_index(drop=True)

  rm = []
  x = []
  for idx, row in df.iterrows():
    pr = row['Positive Reviews']
    nr = row['Negative Reviews']
    total = pr + nr
    if total == 0:
      rm.append(idx)
      total = 1
    rating = (pr - nr)/total * np.log(total)
    x.append(rating)

  df['x'] = x

  df = df.drop(rm)
  df = df.reset_index(drop=True)
 
  x = np.array(x)
  y = np.array(y)
  return (df,x,y)

# Training and Testing

In [80]:
!pip install tensorflow_probability

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
from keras.models import Model
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import *
import os.path
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import tensorflow_probability as tfp
import math

In [82]:
path="/content/steam_games.csv"
df,x,y=getDataset(path)

In [83]:
df.shape

(55497, 6)

In [84]:
x = df['x']
x = np.asarray(x).astype('float32')

In [85]:
y = []
for idx, row in enumerate(df['y']):
    # temp = np.zeros(500)
    temp = []
    for tag in row:
        temp.append(tag)
    y.append(temp)

y = np.array(y).astype('int')

In [86]:
y_train, y_test, x_train, x_test = train_test_split(y, x, test_size=0.2, random_state=42)
print(y_train)

[[1 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [87]:
def customLoss(x, pair):
    tfd = tfp.distributions
    dist = tfd.Normal(loc=pair[0], scale=pair[1])
    loss = tf.reduce_mean(-dist.log_prob(x))
    return loss

In [88]:
num=500
if os.path.isfile("./Test_Model/saved_model.pb"):
    model=tf.keras.models.load_model("Test_Model")
else:
    model = keras.Sequential(
    [
        keras.layers.Dense(num, activation='relu', input_shape=(num,)),
        keras.layers.Dense(num/2, activation='relu'),
        #keras.layers.Dense(1, activation='linear')
        keras.layers.Dense(1,activation=lambda x: tf.nn.elu(x) + 1)
    ])
    #mu = tf.layers.dense(inputs=layer, units=1)
    #sigma = tf.layers.dense(inputs=layer, units=1,activation=lambda x: tf.nn.elu(x) + 1)

    inp = Input((num,))
    x = Dense(num, activation='relu')(inp)
    x = Dense(num/2, activation='relu')(x)
    x = Dense(num/2, activation='relu')(x)
    x = Dense(num/2, activation='relu')(x)
    x = Dense(num/2, activation='relu')(x)
    mu = Dense(1, activation='linear')(x)
    sigma = Dense(1,activation=lambda x: tf.nn.elu(x) + 1)(x)

    model = Model(inp, [mu, sigma])


    # compile the model
    model.compile(optimizer='adam', loss=customLoss)

    # train the model
    model.fit(y_train, x_train, epochs=10, batch_size=32, validation_data=(y_test, x_test))

    model.save("Test_Model")


Epoch 1/10
1388/1388 [==============================] - 9s 5ms/step - loss: nan - dense_58_loss: nan - dense_59_loss: 2.1971 - val_loss: nan - val_dense_58_loss: nan - val_dense_59_loss: 2.1719
Epoch 2/10
1388/1388 [==============================] - 7s 5ms/step - loss: nan - dense_58_loss: nan - dense_59_loss: 2.1722 - val_loss: nan - val_dense_58_loss: nan - val_dense_59_loss: 2.1820
Epoch 3/10
1388/1388 [==============================] - 6s 5ms/step - loss: nan - dense_58_loss: nan - dense_59_loss: 2.1693 - val_loss: nan - val_dense_58_loss: nan - val_dense_59_loss: 2.1675
Epoch 4/10
1388/1388 [==============================] - 6s 4ms/step - loss: nan - dense_58_loss: nan - dense_59_loss: 2.1683 - val_loss: nan - val_dense_58_loss: nan - val_dense_59_loss: 2.1681
Epoch 5/10
1388/1388 [==============================] - 6s 5ms/step - loss: nan - dense_58_loss: nan - dense_59_loss: 2.1673 - val_loss: nan - val_dense_58_loss: nan - val_dense_59_loss: 2.1691
Epoch 6/10
1388/1388 [========

In [93]:
results = model.evaluate(y_test, x_test)

predictresutl,predictresut2 = model.predict(y_test)

print('Test loss:', results)
print('Test predict:', np.any(predictresutl < 0))
print('Test predict:', np.asarray(predictresutl))
print('Test predict:', np.asarray(predictresut2))

#accuracy = metrics.accuracy_score(x_test,predictresutl)
accuracy = metrics.mean_squared_error(x_test,predictresutl)
print(accuracy)

347/347 [==============================] - 1s 1ms/step
Test loss: [nan, nan, 2.1680994033813477]
Test predict: True
Test predict: [[-3.942963 ]
 [-4.0397224]
 [-3.96673  ]
 ...
 [-3.4890628]
 [-3.64805  ]
 [-3.3119586]]
Test predict: [[2.0749793]
 [2.1039186]
 [2.086    ]
 ...
 [1.9659655]
 [2.0077672]
 [1.9191288]]
37.689594


In [92]:
mnb = MultinomialNB(fit_prior=True)

mnb.fit(y_train, x_train.astype('int'))

predictY = mnb.predict(y_test)
print("bayes: ")
accuracy = metrics.accuracy_score(x_test.astype('int'),predictY)
print(accuracy)

bayes: 
0.31126126126126125


In [103]:
import scipy.stats
n_mu=0
n_sigma=1
pred=1
1-scipy.stats.norm(0,1).cdf(pred)

0.15865525393145707